# 加速：多線程爬蟲



* 了解知乎 API 使用方式與回傳內容
* 撰寫程式存取 API 且添加標頭

## 作業目標

* 找一個之前實作過的爬蟲改用多線程改寫，比較前後時間的差異。





In [15]:
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [16]:
job_104_url = "https://www.104.com.tw/cust/list/index/?page=1&order=1&mode=s&jobsource=checkc&area=6001008000&indcat=1001001000"

In [17]:
browser = webdriver.Chrome()
browser.get(job_104_url)  # 打開瀏覽器並連到網頁
time.sleep(2)  # delay一段時間等待網頁更新完成

startTime = time.time()

while True:
    time.sleep(3)  # delay一段時間等待網頁更新完成
    html = browser.page_source
    soup = BeautifulSoup(html, 'lxml')
    company_list = soup.find("div", attrs={'id':"company-result"}).find_all("article", attrs={'class':"items"})
    
    #
    # 擷取公司名稱及簡介內容
    #
    for company in company_list:
        # 因為內容太多，我們把爬取到的結果，寫入"company_list.txt"檔案中，稍後再來檢視
        company_name = company.a.string
        f = open("company_list.txt", "a+", encoding='utf-8')
        f.write( company_name + "\n" )  # 寫入公司名稱
        
        company_desc = company.find("p", attrs={'class':"desc"})
        f.write( company_desc.string + "\n" )  # 寫入公司簡介
        f.write( "--------------------------------------------------------------------------------" + "\n" )
        f.close()
    '''
    
    接下來請學員練習，定位到「下一頁」按鈕。(可以利用find_element_by_link_text("xxx")的函式)
    如果還有下一頁，利用Selenium模擬click「下一頁」按鈕的動作。(定位到物件後，利用其click()屬性)
    若沒有下一頁了，離開爬取的流程
    
    '''
    next_button = browser.find_element_by_link_text("下一頁")
    button_status = next_button.get_attribute('class')  # 查看目前按鈕屬性
    if (button_status == "page-next disabled"):  # 檢查是否還有下一頁
        print("==========沒有下一頁了==========")
        break
    
    next_button.click()  # 模擬點選「下一頁」按鈕

finishTime = time.time()
print(finishTime - startTime) # 正常情況的爬蟲所需時間

==========沒有下一頁了==========
113.05100870132446


In [18]:
def crawler(url,end):
    browser = webdriver.Chrome()
    browser.get(url)  # 打開瀏覽器並連到網頁
    time.sleep(2)  # delay一段時間等待網頁更新完成
    
    count = 1
    while True:
        time.sleep(3)  # delay一段時間等待網頁更新完成
        html = browser.page_source
        soup = BeautifulSoup(html, 'lxml')
        company_list = soup.find("div", attrs={'id':"company-result"}).find_all("article", attrs={'class':"items"})
    
        for company in company_list:
            company_name = company.a.string
            f = open("company_list_thread.txt", "a+", encoding='utf-8')
            f.write( company_name + "\n" )  # 寫入公司名稱
        
            company_desc = company.find("p", attrs={'class':"desc"})
            f.write( company_desc.string + "\n" )  # 寫入公司簡介
            f.write( "--------------------------------------------------------------------------------" + "\n" )
            f.close()
            
        next_button = browser.find_element_by_link_text("下一頁")
        
        if (count == end):  # 檢查是否還有下一頁
            print("==========end==========")
            break
    
        next_button.click()  # 模擬點選「下一頁」按鈕
        count = count+1


In [20]:

all_url = ['https://www.104.com.tw/cust/list/index/?page=1&order=1&mode=s&jobsource=checkc&area=6001008000&indcat=1001001000',
           'https://www.104.com.tw/cust/list/index/?page=14&order=1&mode=s&jobsource=checkc&area=6001008000&indcat=1001001000']

t1 = threading.Thread(target = crawler, args = (all_url[0],13))
t2 = threading.Thread(target = crawler, args = (all_url[1],26))

startTime = time.time()

t1.start()
t2.start()
t1.join()
t2.join()

finishTime = time.time()
print(finishTime - startTime) # 多執行續的爬蟲所需時間

==========end==========
==========end==========
124.24056005477905


## 範例解答##

In [16]:
import requests
import time

urls = ['http://python.org', 'http://python.org', 'http://python.org',
        'http://python.org', 'http://python.org']

startTime = time.time()

for url in urls:
    requests.get(url)

finishTime = time.time()
print(finishTime - startTime) # 正常情況的爬蟲所需時間

8.584307670593262


In [15]:
import requests
import _thread
import time

urls = ['http://python.org', 'http://python.org', 'http://python.org',
        'http://python.org', 'http://python.org']

startTime = time.time()

for url in urls:
    _thread.start_new_thread( requests.get, (url, ) )

finishTime = time.time()
print(finishTime - startTime) # 利用 thread 的爬蟲所需時間

0.0019943714141845703
